In [ ]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import requests as r
from bs4 import BeautifulSoup
from math import ceil
import time
from selenium import webdriver

Getting URL

In [ ]:
url = "https://www.meesho.com/sarees/pl/3iy"

In [ ]:
page_content = r.get(url).content

## Parsing the HTML

In [ ]:
soup = BeautifulSoup(page_content, 'html.parser')
#print(soup.prettify())

In [ ]:
all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))
all_class

[('sc-dkrFOg', 'ProductList__GridCol-sc-8lnc8o-0', 'cokuZA', 'bGpzcx'),
 ('sc-dkrFOg', 'ProductList__GridCol-sc-8lnc8o-0', 'cokuZA', 'eCJiSA'),
 ('sc-dkrFOg', 'ProductList__GridCol-sc-8lnc8o-0', 'cokuZA', 'jGAjKy'),
 ('sc-dkrFOg', 'ProductList__GridCol-sc-8lnc8o-0', 'cokuZA', 'ihwYlH')]

In [ ]:
import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))





In [ ]:
all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_saree= pd.DataFrame(all_data)
df_saree

,Product Name,Ratings,Current price,Previous price,No of Review
0,ETP SATIN SILK SAREES,3.9,238,263,108849
1,TRENDY VOGUISH SAREES,3.7,515,515,177
2,ALISHA FABULOUS SAREES,4.0,1,1,0
3,JIVIKA FASHIONABLE SAREES,4.1,572,572,2206
4,AAGYEYI ATTRACTIVE SAREES,3.8,244,279,45819
5,ABHISARIKA ALLURING SAREES,4.0,260,290,18469
6,ABHISARIKA GEORGETTE SAREES,3.9,289,306,25714
7,+10 MOREJIVIKA FABULOUS SAREES,3.8,550,611,70
8,CHITRAREKHA SENSATIONAL SAREES,3.8,589,589,1571
9,ALISHA GEORGETTE SAREES,3.7,233,251,29935


In [ ]:
df_saree["Discount %"] = round(100- (df_saree["Current price"]/df_saree['Previous price'])*100)

In [ ]:
df_saree

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,ETP SATIN SILK SAREES,3.9,238,263,108849,10.0
1,TRENDY VOGUISH SAREES,3.7,515,515,177,0.0
2,ALISHA FABULOUS SAREES,4.0,1,1,0,0.0
3,JIVIKA FASHIONABLE SAREES,4.1,572,572,2206,0.0
4,AAGYEYI ATTRACTIVE SAREES,3.8,244,279,45819,13.0
5,ABHISARIKA ALLURING SAREES,4.0,260,290,18469,10.0
6,ABHISARIKA GEORGETTE SAREES,3.9,289,306,25714,6.0
7,+10 MOREJIVIKA FABULOUS SAREES,3.8,550,611,70,10.0
8,CHITRAREKHA SENSATIONAL SAREES,3.8,589,589,1571,0.0
9,ALISHA GEORGETTE SAREES,3.7,233,251,29935,7.0


## Men's ware

In [ ]:
url = "https://www.meesho.com/tshirts-men/pl/3k8"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

In [ ]:
import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))


In [ ]:
all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_tshirt= pd.DataFrame(all_data)
df_tshirt

,Product Name,Ratings,Current price,Previous price,No of Review
0,STI MEN'S TSHIRTS,4.0,184,499,6203
1,+11 MORETRENDY NEW SWEATSHIRTS,₹7,115,773,0
2,TRENDY FASHIONISTA MEN TSHIRTS(YELLOW),3.8,298,312,4752
3,FLYIND VOGUE OUTFIT MEN'S TSHIRTS,3.6,384,999,500
4,TRENDY MODERN MEN TSHIRTS,3.8,259,299,39249
5,FANCY RETRO MEN TSHIRTS,3.9,188,203,19356
6,DENZOLEE MEN'S TSHIRTS,3.7,166,205,14720
7,+11 MORETRENDY TSHIRTS,3.9,152,159,145
8,URBANE FASHIONABLE MEN TSHIRTS,3.6,233,261,14727
9,+11 MORENEW TRENDY TSHIRTS,3.9,191,191,254


In [ ]:
df_tshirt["Discount %"] = round(100- (df_tshirt["Current price"]/df_tshirt['Previous price'])*100)
df_tshirt

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,STI MEN'S TSHIRTS,4.0,184,499,6203,63.0
1,+11 MORETRENDY NEW SWEATSHIRTS,₹7,115,773,0,85.0
2,TRENDY FASHIONISTA MEN TSHIRTS(YELLOW),3.8,298,312,4752,4.0
3,FLYIND VOGUE OUTFIT MEN'S TSHIRTS,3.6,384,999,500,62.0
4,TRENDY MODERN MEN TSHIRTS,3.8,259,299,39249,13.0
5,FANCY RETRO MEN TSHIRTS,3.9,188,203,19356,7.0
6,DENZOLEE MEN'S TSHIRTS,3.7,166,205,14720,19.0
7,+11 MORETRENDY TSHIRTS,3.9,152,159,145,4.0
8,URBANE FASHIONABLE MEN TSHIRTS,3.6,233,261,14727,11.0
9,+11 MORENEW TRENDY TSHIRTS,3.9,191,191,254,0.0


In [ ]:
url = "https://www.meesho.com/shirts-men/pl/3jq"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_shirt= pd.DataFrame(all_data)
df_shirt["Discount %"] = round(100- (df_shirt["Current price"]/df_shirt['Previous price'])*100)
df_shirt

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,URBANE FABULOUS MEN SHIRTS,3.7,303,344,3779,12.0
1,TRENDY SENSATIONAL MEN SHIRTS,3.8,215,241,40600,11.0
2,COMFY MODERN MEN SHIRTS,3.5,251,316,35,21.0
3,PRETTY FASHIONISTA MEN SHIRTS,3.3,267,267,4,0.0
4,STI MEN'S SHIRTS,4.0,213,999,43962,79.0
5,ETHNIC TRENDZ MEN'S SHIRTS,4.1,310,355,9114,13.0
6,FANCY LATEST MEN SHIRTS,3.9,404,474,49529,15.0
7,STYLISH PARTYWEAR MEN SHIRTS,3.9,266,266,0,0.0
8,CLASSY FASHIONABLE MEN SHIRTS,3.7,260,295,489,12.0
9,TRENDY RETRO MEN SHIRTS,4.1,219,237,4243,8.0


In [ ]:
url = "https://www.meesho.com/track-pants-men/pl/3kw"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_track= pd.DataFrame(all_data)
df_track["Discount %"] = round(100- (df_track["Current price"]/df_track['Previous price'])*100)
df_track

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,ZEFFIT MEN'S TRACK PANTS,4.0,287,999,18264,71.0
1,STYLISH UNIQUE MEN TRACK PANTS,3.8,554,554,30,0.0
2,RAVISHING FASHIONISTA MEN TRACK PANTS,3.7,642,642,0,0.0
3,CASUAL TRENDY MEN TRACK PANTS,3.9,414,444,131,7.0
4,ELEGANT FABULOUS MEN TRACK PANTS,3.8,263,298,11452,12.0
5,CASUAL GLAMAROUS MEN TRACK PANTS,3.8,290,320,5765,9.0
6,FANCY UNIQUE MEN TRACK PANTS,3.9,305,349,7538,13.0
7,FASHIONABLE FASHIONISTA MEN TRACK PANTS,4.5,743,743,6,0.0
8,RAVISHING FABULOUS MEN TRACK PANTS,3.8,642,642,5,0.0
9,STYLISH FASHIONISTA MEN TRACK PANTS,3.8,278,334,6744,17.0


In [ ]:
url = "https://www.meesho.com/trousers-men/pl/3lw"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df= pd.DataFrame(all_data)
df["Discount %"] = round(100- (df["Current price"]/df['Previous price'])*100)
df

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,CASUAL UNIQUE MEN TROUSER,3.6,197,228,35203,14.0
1,+17 MORESAM & JACK MEN'S TROUSERS,4.0,454,503,10381,10.0
2,DESIGNER TRENDY MEN TROUSERS,3.6,250,276,573,9.0
3,DESIGNER MODERN MEN TROUSERS,3.5,281,314,26,11.0
4,ELEGANT TRENDY MEN TROUSERS,3.7,227,260,7148,13.0
5,FASHIONABLE LATEST MEN TROUSERS,4.0,329,373,17607,12.0
6,ELEGANT GLAMAROUS MEN TROUSERS,4.0,639,701,6915,9.0
7,FASHIONABLE MODERN MEN TROUSERS,3.8,880,880,0,0.0
8,CASUAL GLAMAROUS MEN TROUSERS,3.9,362,362,0,0.0
9,CASUAL MODERN MEN TROUSERS,3.8,311,361,18423,14.0


In [ ]:
url = "https://www.meesho.com/men-watches/pl/3k7"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_watch= pd.DataFrame(all_data)
df_watch["Discount %"] = round(100- (df_watch["Current price"]/df_watch['Previous price'])*100)
df_watch

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,FREE GIFT CLASSIC MEN WATCHES,4.0,191,191,275,0.0
1,CLASSY MEN WATCHES,4.0,219,225,22,3.0
2,STYLISH MEN WATCHES,3.9,330,358,32,8.0
3,STYLISH MEN WATCHES,3.8,139,149,1194,7.0
4,STYLISH MEN WATCHES,3.8,136,142,578,4.0
5,GORGEOUS MEN ANALOG WATCHES,4.1,183,183,2491,0.0
6,UNIQUE MEN WATCHES,3.8,192,201,442,4.0
7,TRENDY VINTAGE METAL MEN'S ANALOG WATCHES VOL 2,3.5,237,263,43,10.0
8,STYLISH MEN ANALOG WATCHES,3.9,152,169,1086,10.0
9,LIMESTONE TRENDY MEN WATCHES,4.1,285,334,187,15.0


In [ ]:
url = "https://www.meesho.com/belts-men/pl/3nn"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_belt= pd.DataFrame(all_data)
df_belt["Discount %"] = round(100- (df_belt["Current price"]/df_belt['Previous price'])*100)
df_belt

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,FASHIONABLE TRENDY MEN BELTS,3.7,125,135,1630,7.0
1,FASHIONABLE LATEST MEN BELTS,4.0,162,175,1560,7.0
2,STYLES TRENDY MEN BELTS,3.6,127,139,1900,9.0
3,FANCY LATEST MEN BELTS,3.4,249,261,0,5.0
4,FASHIONABLE UNIQUE MEN BELTS,4.0,211,234,27073,10.0
5,STYLES TRENDY MEN BELTS,4.1,147,154,4222,5.0
6,FANCY TRENDY MEN BELTS,3.6,130,140,1757,7.0
7,CASUAL UNIQUE MEN BELTS,3.9,124,136,5067,9.0
8,CASUAL TRENDY MEN BELTS,4.0,113,125,2655,10.0
9,STYLES TRENDY MEN BELTS,3.7,96,105,22125,9.0


In [ ]:
url = "https://www.meesho.com/wallets-men/pl/3o5"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_wallet= pd.DataFrame(all_data)
df_wallet["Discount %"] = round(100- (df_wallet["Current price"]/df_wallet['Previous price'])*100)
df_wallet

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,STYLESLATEST MEN WALLETS,3.7,97,105,7311,8.0
1,FASHIONABLEUNIQUE MEN WALLETS,4.0,226,253,6276,11.0
2,FANCYUNIQUE MEN WALLETS,3.7,375,420,0,11.0
3,TRENDY WALLETS,3.9,156,156,0,0.0
4,FASHIONABLETRENDY MEN WALLETS,3.6,94,102,2065,8.0
5,STYLESLATEST MEN WALLETS,4.0,141,157,1667,10.0
6,CASUALUNIQUE MEN WALLETS,4.0,125,140,27059,11.0
7,STYLESLATEST MEN WALLETS,3.9,249,249,42,0.0
8,FASHIONABLETRENDY MEN WALLETS,3.9,97,102,4512,5.0
9,STYLESUNIQUE MEN WALLETS,3.6,95,103,3285,8.0


In [ ]:
url = "https://www.meesho.com/sunglasses-men/pl/3jr"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_sunglass= pd.DataFrame(all_data)
df_sunglass["Discount %"] = round(100- (df_sunglass["Current price"]/df_sunglass['Previous price'])*100)
df_sunglass

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,STYLES MODERN MEN SUNGLASSES,4.1,123,130,215,5.0
1,MEN RECTANGULAR SUNGLASSES,4.0,186,202,605,8.0
2,MEN ROUND SUNGLASSES,4.0,224,259,11827,14.0
3,MEN AVIATOR SUNGLASSES,,292,292,0,0.0
4,MEN SQUARE SUNGLASSES,4.1,142,149,2373,5.0
5,MEN AVIATOR SUNGLASSES,4.0,138,152,3564,9.0
6,CASUAL MODERN MEN SUNGLASSES,4.0,118,126,584,6.0
7,MEN OVAL SUNGLASSES,,292,292,0,0.0
8,MEN AVIATOR SUNGLASSES,3.7,618,618,0,0.0
9,MEN WAYFARER SUNGLASSES,4.0,143,150,1686,5.0


In [ ]:
url = "https://www.meesho.com/sunglasses-men/pl/3jr"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_sunglass= pd.DataFrame(all_data)
df_sunglass["Discount %"] = round(100- (df_sunglass["Current price"]/df_sunglass['Previous price'])*100)
df_sunglass

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,STYLES MODERN MEN SUNGLASSES,4.1,123,130,215,5.0
1,MEN RECTANGULAR SUNGLASSES,4.0,186,202,605,8.0
2,MEN ROUND SUNGLASSES,4.0,224,259,11827,14.0
3,MEN AVIATOR SUNGLASSES,,292,292,0,0.0
4,MEN SQUARE SUNGLASSES,4.1,142,149,2373,5.0
5,MEN AVIATOR SUNGLASSES,4.0,138,152,3564,9.0
6,CASUAL MODERN MEN SUNGLASSES,4.0,118,126,584,6.0
7,MEN OVAL SUNGLASSES,,292,292,0,0.0
8,MEN AVIATOR SUNGLASSES,3.7,618,618,0,0.0
9,MEN WAYFARER SUNGLASSES,4.0,143,150,1686,5.0


In [ ]:
url = "https://www.meesho.com/bags-backpacks-men/pl/3kr"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_bag= pd.DataFrame(all_data)
df_bag["Discount %"] = round(100- (df_bag["Current price"]/df_bag['Previous price'])*100)
df_bag

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,RAVISHING CLASSY WOMEN BACKPACKS,3.8,193,209,4202,8.0
1,ELITE CLASSY WOMEN BACKPACKS,4.1,206,227,5282,9.0
2,CLASSIC STYLISH WOMEN BACKPACKS,4.0,234,259,19641,10.0
3,VOGUISH ALLURING WOMEN BACKPACKS,4.1,371,371,0,0.0
4,VOGUISH FANCY WOMEN BACKPACKS,3.9,191,201,8745,5.0
5,ELEGANT VERSATILE WOMEN BACKPACKS,4.0,193,202,1151,4.0
6,RAVISHING FANCY WOMEN BACKPACKS,3.7,206,217,1657,5.0
7,ELEGANT FASHIONABLE WOMEN BACKPACKS,3.2,213,223,5,4.0
8,VOGUISH ATTRACTIVE WOMEN BACKPACKS,4.1,419,444,0,6.0
9,CLASSIC FANCY WOMEN BACKPACKS,4.1,209,219,3023,5.0


In [ ]:
url = "https://www.meesho.com/casual-shoes-men/pl/3m2"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_shoes= pd.DataFrame(all_data)
df_shoes["Discount %"] = round(100- (df_shoes["Current price"]/df_shoes['Previous price'])*100)
df_shoes

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,RELAXED GRACEFUL MEN CASUAL SHOES,3.4,226,267,18,15.0
1,ASIAN CASUAL SHOES FOR MEN,3.8,583,129,59,-352.0
2,FOOTOX CASUAL SHOES FOR MEN,3.2,367,410,9,10.0
3,UNIQUE TRENDY MEN CASUAL SHOES,3.6,377,377,59,0.0
4,MODERN TRENDY MEN CASUAL SHOES,3.7,247,296,6325,17.0
5,LATEST TRENDY MEN CASUAL SHOES,3.8,205,242,1678,15.0
6,UNIQUE FASHIONABLE MEN CASUAL SHOES,3.6,232,274,183,15.0
7,ASIAN CASUAL SHOES FOR MEN,3.8,500,899,71,44.0
8,FOOTOX CASUAL SHOES FOR MEN,3.9,300,336,8,11.0
9,RAYSFIELD LIFE STYLE CASUAL SHOES FOR MEN,4.0,331,374,14304,11.0


In [ ]:
url = "https://www.meesho.com/sports-shoes-men/pl/3kj"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_sports_shoes= pd.DataFrame(all_data)
df_sports_shoes["Discount %"] = round(100- (df_sports_shoes["Current price"]/df_sports_shoes['Previous price'])*100)
df_sports_shoes

In [ ]:
url = "https://www.meesho.com/sandals-men/pl/3js"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_sandles= pd.DataFrame(all_data)
df_sandles["Discount %"] = round(100- (df_sandles["Current price"]/df_sandles['Previous price'])*100)
df_sandles

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,MODERN FABULOUS MEN SANDALS,3.8,379,379,7450,0.0
1,BUCIK SANDALS FOR MEN,4.0,682,756,11,10.0
2,MODERN TRENDY MEN SANDALS,3.9,271,307,1317,12.0
3,DENSITY SANDALS FOR MEN,3.8,420,499,1285,16.0
4,AKIKO SANDALS FOR MEN,3.8,339,512,50617,34.0
5,MODERN FASHIONABLE MEN SANDALS,3.7,310,377,11907,18.0
6,LATEST FABULOUS MEN SANDALS,3.8,209,248,7449,16.0
7,BUCIK SANDALS FOR MEN,3.8,647,718,4275,10.0
8,TRENDY SANDALS,4.2,187,209,18,11.0
9,LATEST FASHIONABLE MEN SANDALS,3.9,347,384,5755,10.0


In [ ]:
url = "https://www.meesho.com/men-kurtas/pl/3j2"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_kurta= pd.DataFrame(all_data)
df_kurta["Discount %"] = round(100- (df_kurta["Current price"]/df_kurta['Previous price'])*100)
df_kurta

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,COMFY MEN KURTA SETS,4.2,372,408,5138,9.0
1,TRENDY GLAMOROUS MEN SHIRTS,4.1,284,318,2825,11.0
2,ESSENTIAL MEN KURTAS,4.0,295,295,2200,0.0
3,TRENDY MEN KURTA SETS,4.1,382,382,11561,0.0
4,ETHNIC MEN KURTAS,3.9,335,375,4310,11.0
5,UNIQUE MEN KURTAS,3.9,248,290,10540,14.0
6,CLASSY MEN KURTAS,4.1,234,262,7693,11.0
7,URBANE MEN KURTAS,3.8,252,252,2832,0.0
8,TRENDY RETRO COUPLE KURTA SETS,3.8,608,639,2475,5.0
9,ESSENTIAL MEN KURTAS,3.7,383,415,512,8.0


In [ ]:
url = "https://www.meesho.com/ethnic-jackets-men/pl/3j6"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_jacket= pd.DataFrame(all_data)
df_jacket["Discount %"] = round(100- (df_jacket["Current price"]/df_jacket['Previous price'])*100)
df_jacket

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,URBANE MEN ETHNIC JACKETS,3.8,415,461,1203,10.0
1,CLASSIC MEN ETHNIC JACKETS,4.2,366,406,915,10.0
2,STYLEVAULT MEN'S ETHNIC JACKETS,4.0,462,485,1388,5.0
3,DESIGNER MEN ETHNIC JACKETS,3.9,384,384,0,0.0
4,MODERN MEN ETHNIC JACKETS,4.0,341,395,4331,14.0
5,ELEGANT MEN ETHNIC JACKETS,4.1,324,362,28244,10.0
6,FANCY MEN ETHNIC JACKETS,4.1,344,394,3219,13.0
7,CLASSIC MEN ETHNIC JACKETS,4.0,366,424,190,14.0
8,CLASSIC MEN ETHNIC JACKETS,4.1,353,390,2533,9.0
9,FASHIONABLE MEN ETHNIC JACKETS,3.9,400,434,445,8.0


In [ ]:
url = "https://www.meesho.com/dresses-kids/pl/3jg"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_kids_dress= pd.DataFrame(all_data)
df_kids_dress["Discount %"] = round(100- (df_kids_dress["Current price"]/df_kids_dress['Previous price'])*100)
df_kids_dress

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,CUTE STYLUS GIRLS FROCKS & DRESSES,3.9,228,258,10109,12.0
1,PRETTY FANCY GIRLS FROCKS & DRESSES,4.1,88,946,4258,91.0
2,CUTIEPIE FUNKY GIRLS FROCKS & DRESSES,3.9,149,149,0,0.0
3,MODERN COMFY GIRLS FROCKS & DRESSES,3.8,290,305,1354,5.0
4,ELEGANT FLARED GIRLS FROCKS & DRESSES,3.8,89,923,28955,90.0
5,PRETTY FUNKY GIRLS FROCKS & DRESSES,4.0,155,168,5491,8.0
6,FLAWSOME FANCY GIRLS FROCKS & DRESSES,3.8,123,132,17103,7.0
7,CUTE CLASSY GIRLS FROCKS & DRESSES,3.8,295,309,1457,5.0
8,MODERN TRENDY GIRLS FROCKS & DRESSES,4.0,103,114,84611,10.0
9,TINKLE CLASSY GIRLS FROCKS & DRESSES,4.2,141,148,18346,5.0


In [ ]:
url = "https://www.meesho.com/rompers/pl/3l4"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_infants= pd.DataFrame(all_data)
df_infants["Discount %"] = round(100- (df_infants["Current price"]/df_infants['Previous price'])*100)
df_infants

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,PRINCESS FANCY BOYS ONESIES & ROMPERS,4.3,326,326,1000,0.0
1,PRETTY STYLUS BOYS ONESIES & ROMPERS,4.1,279,302,1406,8.0
2,PRETTY ELEGANT BOYS ONESIES & ROMPERS,4.1,349,378,2988,8.0
3,MODERN TRENDY BOYS ONESIES & ROMPERS,4.2,469,545,467,14.0
4,TINKLE ELEGANT BOYS ONESIES & ROMPERS,4.1,389,408,6029,5.0
5,PRETTY FUNKY BOYS ONESIES & ROMPERS,4.2,389,408,3986,5.0
6,CUTIEPIE ELEGANT BOYS ONESIES & ROMPERS,4.0,188,188,1675,0.0
7,PRINCESS COMFY BOYS ONESIES & ROMPERS,4.3,371,389,775,5.0
8,AGILE ELEGANT BOYS ONESIES & ROMPERS,4.1,337,353,2211,5.0
9,CUTE CLASSY BOYS ONESIES & ROMPERS,4.2,336,336,822,0.0


In [ ]:
url = "https://www.meesho.com/soft-toys-kids/pl/3l1"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_toy= pd.DataFrame(all_data)
df_toy["Discount %"] = round(100- (df_toy["Current price"]/df_toy['Previous price'])*100)
df_toy

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,FASHIONATE UNISEX SOFT TOYS,3.8,224,248,306,10.0
1,FASHIONATE UNISEX SOFT TOYS,3.8,216,227,2620,5.0
2,ATTRACTIVE UNISEX SOFT TOYS,3.9,263,263,535,0.0
3,CASUAL KIDS SOFT TOYS,4.1,172,187,12326,8.0
4,STYLO ELECTRONIC PETS,3.8,278,278,4016,0.0
5,FABULOUS UNISEX SOFT TOYS,3.9,205,243,9962,16.0
6,CLASSY UNISEX SOFT TOYS,3.7,242,268,2039,10.0
7,SOFT TOYS,3.7,244,285,2002,14.0
8,GRACEFUL KIDS SOFT TOY,4.3,142,155,5983,8.0
9,ATTRACTIVE UNISEX SOFT TOYS,3.7,245,283,6220,13.0


In [ ]:
url = "https://www.meesho.com/footwear-kids/pl/3ti"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_kids_footware= pd.DataFrame(all_data)
df_kids_footware["Discount %"] = round(100- (df_kids_footware["Current price"]/df_kids_footware['Previous price'])*100)
df_kids_footware

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,STYLO KIDS BOYS KIDS BOYS FLIP FLOPS,4.0,190,214,3714,11.0
1,NEW TRENDY FLIP FLOPS,3.8,158,184,25,14.0
2,FASHIONABLE KIDS BOYS KIDS BOYS FLIP FLOPS,4.0,125,125,0,0.0
3,FANCY KIDS BOYS KIDS BOYS FLIP FLOPS,3.8,191,191,1072,0.0
4,UNIQUE KIDS BOYS KIDS BOYS CASUAL SHOES,4.1,286,312,9533,8.0
5,ATTRACTIVE STYLISH KIDS GIRLS FLIP FLOPS,4.0,166,183,7477,9.0
6,MODERN KIDS BOYS KIDS BOYS SPORTS SHOES,4.1,264,296,3284,11.0
7,WONDERFUL KIDS BOYS KIDS BOYS SPORTS SHOES,₹8,399,854,0,53.0
8,PRETTY CLASSY KIDS GIRLS FLIP FLOPS,4.0,181,201,741,10.0
9,CUTIEPIE TRENDY KIDS GIRLS CASUAL SHOES,4.2,361,400,4580,10.0


In [ ]:
url = "https://www.meesho.com/watches-kids/pl/3ko"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_sat= pd.DataFrame(all_data)
df_sat["Discount %"] = round(100- (df_sat["Current price"]/df_sat['Previous price'])*100)
df_sat

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,STYLISH KIDS UNISEX WATCHES,4.2,145,154,1907,6.0
1,FASHIONABLE WATCHES,3.6,132,150,113,12.0
2,FABULOUS KIDS UNISEX WATCHES,3.8,220,231,133,5.0
3,ELITE WATCHES,4.1,166,182,667,9.0
4,IMAGES PROJECTOR DIGITAL TOY WATCH FOR KIDS,4.1,153,165,264,7.0
5,NEW WATCHES,3.8,165,178,152,7.0
6,FUNKY KIDS UNISEX WATCHES,4.1,147,167,257,12.0
7,FASHIONATE KIDS UNISEX WATCHES,3.9,167,185,562,10.0
8,ELITE KIDS UNISEX WATCHES,4.0,145,165,131,12.0
9,GORGEOUS WOMEN WATCHES,4.2,148,164,38,10.0


In [ ]:
url = "https://www.meesho.com/bags-kids/pl/3kx"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_kids_bag= pd.DataFrame(all_data)
df_kids_bag["Discount %"] = round(100- (df_kids_bag["Current price"]/df_kids_bag['Previous price'])*100)
df_kids_bag

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,COMFY CLASSIC KIDS UNISEX BAGS & BACKPACKS,3.6,257,278,3254,8.0
1,COMFY FASHIONABLE KIDS UNISEX BAGS & BACKPACKS,3.9,196,216,1530,9.0
2,FANCY FASHIONABLE KIDS UNISEX BAGS & BACKPACKS,4.0,200,200,0,0.0
3,COMFY FASHIONABLE KIDS UNISEX BAGS & BACKPACKS,4.1,268,278,1346,4.0
4,ELITE KIDS BAGS & BACKPACKS,3.9,233,270,15223,14.0
5,FUNKY FASHIONABLE KIDS UNISEX BAGS & BACKPACKS,4.0,184,203,8466,9.0
6,FUNKY DESIGNER KIDS UNISEX BAGS & BACKPACKS,3.9,224,259,6360,14.0
7,LATEST DESIGNER KIDS UNISEX BAGS & BACKPACKS,3.8,334,334,16,0.0
8,STYLISH MODERN KIDS UNISEX BAGS & BACKPACKS,3.7,213,233,6897,9.0
9,STYLISH FASHIONATE KIDS UNISEX BAGS & BACKPACKS,3.8,177,200,2068,12.0


In [ ]:
url = "https://www.meesho.com/baby-care/pl/3tj"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_baby_care= pd.DataFrame(all_data)
df_baby_care["Discount %"] = round(100- (df_baby_care["Current price"]/df_baby_care['Previous price'])*100)
df_baby_care

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,CLASSIC BABY BLANKET,4.0,385,413,1100,7.0
1,TRENDY BABY BLANKET,4.2,208,213,11053,2.0
2,ATTRACTIVE BABY BLANKET,3.8,191,209,489,9.0
3,TRENDY BABY BLANKET,4.0,216,216,1012,0.0
4,ATTRACTIVE BABY BLANKET,4.0,140,160,13937,12.0
5,ELEGANT BABY TOWELS,4.2,97,103,21196,6.0
6,CLASSIC BABY TOWELS,4.1,162,184,9966,12.0
7,TRENDY BABY BLANKET,4.2,256,256,316,0.0
8,GORGEOUS BABY TOWELS,4.3,120,129,16866,7.0
9,FANCY BABY BLANKET,4.1,226,239,926,5.0


In [ ]:
full_data = pd.concat([df_saree,df_tshirt,df_shirt,df_watch,df_belt,df_wallet,df_sunglass,df_bag,df_shoes,df_sports_shoes,df_sandles,df_kurta, df_jacket,df_kids_dress,df_infants,df_toy,df_kids_footware,df_sat,df_kids_bag,df_baby_care],ignore_index=True)
full_data

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,ETP SATIN SILK SAREES,3.9,238,263,108849,10.0
1,TRENDY VOGUISH SAREES,3.7,515,515,177,0.0
2,ALISHA FABULOUS SAREES,4.0,1,1,0,0.0
3,JIVIKA FASHIONABLE SAREES,4.1,572,572,2206,0.0
4,AAGYEYI ATTRACTIVE SAREES,3.8,244,279,45819,13.0
...,...,...,...,...,...,...
394,COMFY BABY BLANKET,4.0,402,402,101,0.0
395,FASHIONABLE BLANKET,3.7,147,153,7215,4.0
396,RAVISHING BABY MOSQUITO NETS,3.9,203,213,6790,5.0
397,COMFY BABY TOWELS,4.3,192,197,4435,3.0


In [ ]:
full_data.to_csv("Mens_Kids_data.csv")

## Home Appliance

In [ ]:
url = "https://www.meesho.com/bedsheets/pl/3jm"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_bedsheet= pd.DataFrame(all_data)
df_bedsheet["Discount %"] = round(100- (df_bedsheet["Current price"]/df_bedsheet['Previous price'])*100)
df_bedsheet

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,ALLURING BEDSHEETS,4.0,220,230,8879,4.0
1,TRENDY BEDSHEETS,3.8,187,203,12884,8.0
2,GORGEOUS BEDSHEETS,4.0,196,205,6708,4.0
3,FASHIONABLE BEDSHEETS,3.6,507,507,0,0.0
4,VOGUISH BEDSHEETS,4.3,413,434,59262,5.0
5,ATTRACTIVE BEDSHEETS,4.1,229,229,18444,0.0
6,VOGUISH BEDSHEETS,3.7,380,399,27944,5.0
7,VOGUISH BEDSHEETS,3.8,221,231,8643,4.0
8,ALLURING BEDSHEETS,3.8,207,217,7885,5.0
9,ELEGANT BEDSHEETS,3.7,183,202,9055,9.0


In [ ]:
url = "https://www.meesho.com/door-bath-mats/pl/3kf"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_door_mat= pd.DataFrame(all_data)
df_door_mat["Discount %"] = round(100- (df_door_mat["Current price"]/df_door_mat['Previous price'])*100)
df_door_mat

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,KITCHEN MAT,3.9,259,287,239,10.0
1,NEW KITCHEN MAT,3.6,206,244,213,16.0
2,RAVISHING VERSATILE DOORMATS,3.2,201,238,43,16.0
3,ELITE KITCHEN MAT,3.5,166,180,8144,8.0
4,COLORFUL KITCHEN MAT,4.0,118,123,5448,4.0
5,TRENDY KITCHEN MAT,3.9,111,115,9860,3.0
6,WONDERFUL KITCHEN MAT,4.2,220,230,2243,4.0
7,MODERN KITCHEN MAT,4.4,310,347,705,11.0
8,GRACEFUL FRIDGE MATS,3.5,109,121,776,10.0
9,VOGUISH STYLISH DOORMATS,4.0,207,233,480,11.0


In [ ]:
url = "https://www.meesho.com/curtains-sheers/pl/3jn"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_curtains= pd.DataFrame(all_data)
df_curtains["Discount %"] = round(100- (df_curtains["Current price"]/df_curtains['Previous price'])*100)
df_curtains

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,TRENDY SHOWER CURTAINS,4.2,333,349,1412,5.0
1,DESIGNER CURTAINS,3.6,141,147,8,4.0
2,FASHIONABLE CURTAINS,4.1,310,332,3562,7.0
3,RAVISHING CURTAINS,4.3,257,277,7402,7.0
4,RAVISHING STYLISH CURTAINS & SHEERS,3.8,263,288,3733,9.0
5,CLASSIC CURTAINS,3.7,228,266,11056,14.0
6,RAVISHING CLASSY CURTAINS & SHEERS,4.1,145,148,8544,2.0
7,ALLURING CURTAINS,3.9,259,274,2266,5.0
8,ATTRACTIVE CURTAINS,4.3,386,405,4559,5.0
9,FANCY CURTAINS,4.1,125,137,1327,9.0


In [ ]:
url = "https://www.meesho.com/cushion-covers/pl/3k3"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_cushion_covers= pd.DataFrame(all_data)
df_cushion_covers["Discount %"] = round(100- (df_cushion_covers["Current price"]/df_cushion_covers['Previous price'])*100)
df_cushion_covers

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,STYLISH CUSHION COVERS,3.7,186,186,2548,0.0
1,FASHIONABLE CUSHION COVERS,4.1,191,200,3491,4.0
2,VOGUISH CUSHION COVERS,3.9,239,258,8,7.0
3,ALLURING CUSHION COVERS,4.1,171,185,24616,8.0
4,ATTRACTIVE CUSHION COVERS,4.1,169,183,13757,8.0
5,ELEGANT CUSHION COVERS,4.2,237,249,5768,5.0
6,GRACEFUL ALLURING CUSHION COVERS,4.1,168,175,14945,4.0
7,ATTRACTIVE CUSHION COVERS,4.2,243,254,1578,4.0
8,GRACEFUL CUSHIONS,4.3,459,492,8362,7.0
9,FASHIONABLE CUSHION COVERS,4.1,170,187,4120,9.0


In [ ]:
url = "https://www.meesho.com/mattress-protectors/pl/3l3"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_mattress_protectors = pd.DataFrame(all_data)
df_mattress_protectors["Discount %"] = round(100- (df_mattress_protectors["Current price"]/df_mattress_protectors['Previous price'])*100)
df_mattress_protectors

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,FANCY BEDSHEETS,3.9,170,170,126,0.0
1,CHECK OUT THIS TRENDING CATALOG,4.0,292,323,1019,10.0
2,CHECK OUT THIS TRENDING CATALOG,3.9,269,291,452,8.0
3,GORGEOUS BABY MATS & BED PROTECTOR,4.0,252,264,723,5.0
4,MATTRESS PROTECTOR BABY MAT & BED PROTECTORS,4.2,529,567,2734,7.0
5,MATTRESS PROTECTOR BABY MAT & BED PROTECTORS,4.2,550,590,9414,7.0
6,RAVISHING BEDSHEETS,3.9,229,229,3224,0.0
7,TRENDY MATTRESSES PROTECTORS,2.6,155,155,8,0.0
8,CHECK OUT THIS TRENDING CATALOG,3.7,353,392,911,10.0
9,ATTRACTIVE MATTRESSES PROTECTORS,4.0,315,370,162,15.0


## Home Decore

In [ ]:
url = "https://www.meesho.com/home-decor/pl/3tl"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_home_dec = pd.DataFrame(all_data)
df_home_dec["Discount %"] = round(100- (df_home_dec["Current price"]/df_home_dec['Previous price'])*100)
df_home_dec

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,FANCY HOME DECOR,3.8,608,608,0,0.0
1,HOME DECOR,3.8,443,443,0,0.0
2,CLASSIC DREAM CATCHERS,4.0,327,327,7,0.0
3,CLASSIC DREAM CATCHERS,4.2,357,357,0,0.0
4,FABULOUS DREAM CATCHERS,4.0,327,327,5,0.0
5,DESIGNER DREAM CATCHERS,3.6,362,362,5,0.0
6,HOME LIGHTS,3.9,370,370,0,0.0
7,CLASSIC DECOR,,431,431,0,0.0
8,LATEST DECOR,,910,910,0,0.0
9,HOME DECOR,3.8,394,394,0,0.0


In [ ]:
url = "https://www.meesho.com/clocks/pl/3kh"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_home_dec1 = pd.DataFrame(all_data)
df_home_dec1["Discount %"] = round(100- (df_home_dec1["Current price"]/df_home_dec1['Previous price'])*100)
df_home_dec1

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,GORGEOUS CLOCKS,4.2,231,231,0,0.0
1,TRENDY WALL CLOCK,3.8,191,191,3231,0.0
2,ESSENTIAL WALL CLOCKS,4.2,428,428,8429,0.0
3,GRACEFUL WALL CLOCK,3.8,242,242,787,0.0
4,ATTRACTIVE CLOCKS,3.8,228,234,925,3.0
5,STYLISH ACRYLIC DESIGNER WALL CLOCK VOL 4,3.9,191,204,4503,6.0
6,MODERN WALL CLOCKS,3.8,331,355,7332,7.0
7,DESIGNER KITCHEN TIMERS,4.0,284,284,3536,0.0
8,ATTRACTIVE CLOCKS,3.9,312,334,1040,7.0
9,FANCY WALL CLOCKS,3.9,211,226,5096,7.0


In [ ]:
url = "https://www.meesho.com/show-pieces-home/pl/3ku"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_home_dec2 = pd.DataFrame(all_data)
df_home_dec2["Discount %"] = round(100- (df_home_dec2["Current price"]/df_home_dec2['Previous price'])*100)
df_home_dec2

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,ESSENTIAL IDOLS & FIGURINES,4.0,108,112,1206,4.0
1,TRENDY NEW SHOW PIECES,4.2,154,175,6,12.0
2,UNIQUE IDOLS & FIGURINES,4.2,134,147,298,9.0
3,ATTRACTIVE IDOLS & FIGURINES,4.1,138,145,18953,5.0
4,ATTRACTIVE IDOLS & FIGURINES,4.1,154,167,1465,8.0
5,STYLO IDOLS & FIGURINES,3.9,212,239,950,11.0
6,TRENDY HOME DECOR SHOW PIECES,3.8,255,283,6374,10.0
7,TRENDY NEW SHOW PIECES,4.2,256,268,30,4.0
8,DOON FURNITURE HOUSE COMBOS PACKAGE,4.0,455,494,1308,8.0
9,DIVA STYLISH DESIGNER HOME DECOR VOL 7,4.3,416,437,353,5.0


In [ ]:
url = "https://www.meesho.com/kitchen-storage/pl/3tn"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_kitchen = pd.DataFrame(all_data)
df_kitchen ["Discount %"] = round(100- (df_kitchen["Current price"]/df_kitchen['Previous price'])*100)
df_kitchen

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,UNIQUE JARS & CONTAINERS,4.3,221,221,4961,0.0
1,TRENDY JARS & CONTAINERS,4.0,301,301,32,0.0
2,TRENDY JARS & CONTAINERS,4.0,272,272,79,0.0
3,TRENDY JARS & CONTAINERS,4.0,346,346,6388,0.0
4,DESIGNER JARS & CONTAINERS,4.0,493,540,11833,9.0
5,CLASSY WATER BOTTLES,4.1,172,217,16853,21.0
6,FANCY JARS & CONTAINERS,3.6,274,310,12421,12.0
7,CLASSY JARS & CONTAINERS,3.9,382,382,80,0.0
8,UNIQUE WATER BOTTLES,3.9,382,481,90,21.0
9,WONDERFUL SPICE RACKS,3.6,197,215,3428,8.0


In [ ]:
url = "https://www.meesho.com/cookware-bakeware/pl/3tm"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_kitchen_1 = pd.DataFrame(all_data)
df_kitchen_1["Discount %"] = round(100- (df_kitchen_1["Current price"]/df_kitchen_1['Previous price'])*100)
df_kitchen_1

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,CLASSIC TADKA PANS,4.0,145,159,12358,9.0
1,CLASSY APPAM MAKER,3.8,324,355,6154,9.0
2,DESIGNER SAUCE PANS,4.0,236,271,155,13.0
3,FANCY APPAM MAKER,3.6,284,319,2729,11.0
4,CLASSIC TADKA PANS,3.9,128,156,13374,18.0
5,CLASSIC APPAM MAKER,3.7,274,293,2405,6.0
6,ESSENTIAL FRYING PANS,3.9,240,283,4952,15.0
7,STYLO FAJITA PANS,3.1,261,515,185,49.0
8,FANCY APPAM MAKER,3.7,434,542,1484,20.0
9,CLASSY APPAM MAKER,3.6,250,264,2518,5.0


In [ ]:
full_data_1= pd.concat([df_bedsheet,df_cushion_covers,df_curtains,df_door_mat,df_mattress_protectors,df_home_dec,df_home_dec1,df_home_dec2,df_kitchen,df_kitchen_1],ignore_index=True)
full_data_1

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,ALLURING BEDSHEETS,4.0,220,230,8879,4.0
1,TRENDY BEDSHEETS,3.8,187,203,12884,8.0
2,GORGEOUS BEDSHEETS,4.0,196,205,6708,4.0
3,FASHIONABLE BEDSHEETS,3.6,507,507,0,0.0
4,VOGUISH BEDSHEETS,4.3,413,434,59262,5.0
...,...,...,...,...,...,...
188,ELITE GRILL PANS,3.7,517,557,8705,7.0
189,STYLO APPAM MAKER,3.6,681,820,4792,17.0
190,STYLO APPAM MAKER,3.8,661,758,5587,13.0
191,STYLO SAUCE POTS & HANDIS,3.9,555,583,3965,5.0


In [ ]:
full_data_1.to_csv("Home_appliance_data.csv")

## Beauty and Health

In [ ]:
url = "https://www.meesho.com/face-makeup/pl/3j7"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_beauty_1 = pd.DataFrame(all_data)
df_beauty_1["Discount %"] = round(100- (df_beauty_1["Current price"]/df_beauty_1['Previous price'])*100)
df_beauty_1

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,OZONE SENSATIONAL CLEANSING CLEANSERS,4.2,436,600,492,27.0
1,FANCY MAKEUP KITS,3.6,142,150,4932,5.0
2,EVERYDAY FACE MAKEUP COMBO,3.9,195,204,504,4.0
3,FACES CANADA 2-STEP GLOW GETTER MAKEUP KIT - R...,4.1,918,119,129,-671.0
4,SUPERIOR RICH LOOK LIPSTICKS,3.9,104,109,6846,5.0
5,PROFFESIONAL ABSOLUTE FOUNDATION,4.0,183,196,568,7.0
6,USEFUL FACE MAKEUP COMBO,3.9,164,171,1381,4.0
7,USEFUL FACE MAKEUP COMBO,3.6,119,129,1060,8.0
8,PROFFESIONAL HYDRATING CLEANSERS,4.0,146,182,587,20.0
9,USEFUL FACE MAKEUP COMBO,3.8,208,232,46,10.0


In [ ]:
url = "https://www.meesho.com/eye-makeup/pl/3j1"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_beauty_2 = pd.DataFrame(all_data)
df_beauty_2["Discount %"] = round(100- (df_beauty_2["Current price"]/df_beauty_2['Previous price'])*100)
df_beauty_2

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,USEFUL EYE MAKEUP COMBO,3.9,111,111,0,0.0
1,NEW COLLECTIONS OF EYE MAKEUP COMBO,3.7,218,229,143,5.0
2,NEW EYE MAKEUP COMBO,3.9,313,328,177,5.0
3,TRENDY EYE MAKEUP COMBO,4.1,578,865,0,33.0
4,TRENDY NEW EYE MAKEUP COMBO,4.0,128,140,831,9.0
5,NEW EYE MAKEUP COMBO,3.9,237,243,71,2.0
6,KOREAN EVERYDAY EYE MAKEUP COMBO,3.9,123,138,666,11.0
7,NEW COLLECTIONS OF EYE MAKEUP COMBO,3.9,125,131,2100,5.0
8,CLASSIC EYE MAKEUP COMBO,3.7,132,150,471,12.0
9,CLASSY MAKEUP COMBO,3.8,170,193,88,12.0


In [ ]:
url = "https://www.meesho.com/lips-makeup/pl/3jd"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_beauty_3 = pd.DataFrame(all_data)
df_beauty_3["Discount %"] = round(100- (df_beauty_3["Current price"]/df_beauty_3['Previous price'])*100)
df_beauty_3

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,SUPERIOR UNIQUE LIPSTICKS,4.0,197,215,5438,8.0
1,VLCC ANTI TAN SINGLE FACIAL KIT - 60 G,4.0,263,315,424,17.0
2,NUTRIGLOW ATTRACTIVE NUTRIGLOW NATURALS FACIAL...,4.1,117,150,1489,22.0
3,TRENDY MAKEUP BRUSHES,4.2,167,167,883,0.0
4,PROFFESIONAL RICH LOOK LIPSTICKS,3.8,106,116,7501,9.0
5,"GURIYA, MULTICOLOR/ MATTELIQUID/ LIPSTICKPACK ...",3.8,100,108,54261,7.0
6,SENSATIONAL LONG STAY LIPSTICKS,4.3,256,299,64059,14.0
7,FABULOUS MAKEUP ACCESSORIES,4.4,255,255,1782,0.0
8,FANCY MINARA BEAUTY MAKEUP KITS,4.1,249,249,0,0.0
9,FANCY MAKEUP COMBO,3.7,239,250,3297,4.0


In [ ]:
full_data_2= pd.concat([df_beauty_1,df_beauty_2,df_beauty_3,df_beauty_4,df_beauty_5],ignore_index=True)
full_data_2

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,OZONE SENSATIONAL CLEANSING CLEANSERS,4.2,436.0,600.0,492.0,27.0
1,FANCY MAKEUP KITS,3.6,142.0,150.0,4932.0,5.0
2,EVERYDAY FACE MAKEUP COMBO,3.9,195.0,204.0,504.0,4.0
3,FACES CANADA 2-STEP GLOW GETTER MAKEUP KIT - R...,4.1,918.0,119.0,129.0,-671.0
4,SUPERIOR RICH LOOK LIPSTICKS,3.9,104.0,109.0,6846.0,5.0
...,...,...,...,...,...,...
64,HYGIENE SANITARY PADS,4.0,190.0,199.0,9.0,5.0
65,NEW FEEL SANITARY PAD SANITARY,4.2,219.0,227.0,3002.0,4.0
66,CLASSIC MENSTRUAL/SANITARY PADS,3.9,218.0,218.0,0.0,0.0
67,SANNAP FEMININE HYGIENE PRODUCTS VOL 1,3.0,279.0,279.0,4.0,0.0


In [ ]:
full_data_2.to_csv("Beauty_data.csv")

In [ ]:
url = "https://www.meesho.com/oral-care/pl/3tf"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_beauty_4 = pd.DataFrame(all_data)
df_beauty_4["Discount %"] = round(100- (df_beauty_4["Current price"]/df_beauty_4['Previous price'])*100)
df_beauty_4

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,MOUTH WASH,3.3,138,138,0,0.0
1,ADBUCHHI CLASSY ORAL CARE KITS,3.8,221,221,0,0.0
2,EVERYDAY ORAL CARE KITS,4.1,101,111,66,9.0
3,URBANMOOCH NATURAL ACTIVATED CHARCOAL TEETH WH...,4.0,159,159,917,0.0
4,DENTA 3 TOOTH PASTE WITH MOUTHWASH,4.1,123,126,21,2.0
5,URBAN MOOCH URBANMOOCH FANCY TEETH WHITENING,4.1,148,148,58,0.0
6,ORAYA ACTIVATED CHARCOAL INSTANT TEETH WHITENI...,3.7,117,117,0,0.0
7,AROMINE 100% PURE BABOOL FALI - KIKAR FALI (P...,3.7,167,167,0,0.0
8,AKILLI TEETH WHITENING CHARCOAL ACTIVATED CARB...,4.1,548,548,7,0.0
9,MOUTH WASH,3.3,138,138,0,0.0


In [ ]:
url = "https://www.meesho.com/feminine-hygiene/pl/3tg"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_beauty_5 = pd.DataFrame(all_data)
df_beauty_5["Discount %"] = round(100- (df_beauty_5["Current price"]/df_beauty_5['Previous price'])*100)
df_beauty_5

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,+11 MOREHYGIENE SANITARY PADS,3.4,161,168,5,4.0
1,MIRACLE CHOICE 280/240 MM STRAIGHT NAPKINS WIT...,4.0,102,110,0,7.0
2,KEY SOUL INTIMATE WASH,,353,353,0,0.0
3,NEW FEEL,4.3,162,183,3968,11.0
4,REGULAR NEW FEEL SANITARY PAD,4.1,212,219,13781,3.0
5,SILKY REUSABLE MENSTRUAL CUPS VOL 2,4.0,197,197,1683,0.0
6,EXTRA SURE SANITARY PADS,4.3,324,336,2315,4.0
7,VOLAMENA BODY CARE PRODUCT VOL 10,4.3,125,151,759,17.0
8,????????ARE WOMEN'S HYGIENIC SILICONE CUP VOL 2,4.0,244,244,625,0.0
9,ANION SANITARY PADS,4.4,240,240,79,0.0
